In [1]:
rm(list=ls())
#getwd()
#dir='/Users/sungjinpark/OneDrive - konkuk.ac.kr/datamining/appstore_2/2experiment'
#setwd(dir)

In [2]:
load('reg_1_train.rda')
load('reg_1_test.rda')
levels(reg_1_train$prime_genre)

[1] "Games"     "Non_Games"

# 1.    Data 1

## prime_genre : Games NonGames


- 6measure

      Average Error, MAE(Mean Absolute Error), 
      MAPE(Mean Absolute Percentage Error), MSE, RMSE, R^2_adj
      

- data : reg_1_train & reg_1_test      

## 다중공선성이 의심되는 변수 체크

- rating_count_ver, sum_rate_ver

In [5]:
reg_1_train<-droplevels(reg_1_train)
reg_1_test <- droplevels(reg_1_test)

In [6]:
set.seed(1)
library(car)
lm.fit_1=lm(user_rating_scaled~.,data=reg_1_train)
vif(lm.fit_1)

,GVIF,Df,GVIF^(1/(2*Df))
price,1.052870,1,1.026095
rating_count_tot,1.061510,1,1.030296
rating_count_ver,116.388911,1,10.788369
user_rating_ver,1.163316,1,1.078571
ver,1.221739,1,1.105323
cont_rating,1.241018,3,1.036644
prime_genre,1.556395,1,1.247555
sup_devices.num,1.092360,1,1.045160
ipadSc_urls.num,1.231630,1,1.109788
lang.num,1.090321,1,1.044184


## sum_rate_ver 제거후 VIF 체크

In [7]:
lm.fit_2=lm(user_rating_scaled~.,data=reg_1_train[,-12])
vif(lm.fit_2)

,GVIF,Df,GVIF^(1/(2*Df))
price,1.052870,1,1.026095
rating_count_tot,1.061176,1,1.030134
rating_count_ver,1.032277,1,1.016010
user_rating_ver,1.163312,1,1.078569
ver,1.221314,1,1.105131
cont_rating,1.240621,3,1.036589
prime_genre,1.556203,1,1.247479
sup_devices.num,1.091439,1,1.044720
ipadSc_urls.num,1.231217,1,1.109602
lang.num,1.090212,1,1.044132


## 다중공선성을 야기하는 sum_rate_ver 제거후 진행

In [8]:
drops=c("sum_rate_ver")
reg_1_train=reg_1_train[ , !(names(reg_1_train) %in% drops)]
reg_1_test=reg_1_test[ , !(names(reg_1_test) %in% drops)]


## 1.1 linear-regression

In [9]:
c_data1.mat = matrix(0,4,6)
colnames(c_data1.mat) = c( "AE","MAE","MAPE","MSE","RMSE","R_adj") 
rownames(c_data1.mat) = c("Default_reg","Forward","Backward",'Stepwise')
c_data1.mat

,AE,MAE,MAPE,MSE,RMSE,R_adj
Default_reg,0,0,0,0,0,0
Forward,0,0,0,0,0,0
Backward,0,0,0,0,0,0
Stepwise,0,0,0,0,0,0


In [10]:
reg_1_train<-droplevels(reg_1_train)
reg_1_test <- droplevels(reg_1_test)
####Default####
set.seed(1)
lm.fit_1=lm(user_rating_scaled~.,data=reg_1_train)
#reg_1 predict
set.seed(1)
p.df_1=predict(lm.fit_1,newdata=reg_1_test)
###6Measure###
#average error
ave.err.df_1=mean(reg_1_test$user_rating_scaled-p.df_1)
c_data1.mat[1,1]=ave.err.df_1
#mae
mae.df_1=mean(abs(reg_1_test$user_rating_scaled-p.df_1))
c_data1.mat[1,2]=mae.df_1
#mape
mape.df_1=0.01*mean(abs(reg_1_test$user_rating_scaled-p.df_1)/abs(reg_1_test$user_rating_scaled))
c_data1.mat[1,3]=mape.df_1
#mse
mse.df_1=mean((reg_1_test$user_rating_scaled-p.df_1)^2)
c_data1.mat[1,4]=mse.df_1
#rmse
rmse.df_1=sqrt(mse.df_1)
c_data1.mat[1,5]=rmse.df_1
#adj.r.squared
adj.df_1=summary(lm.fit_1)$adj.r.squared
c_data1.mat[1,6]=adj.df_1
c_data1.mat


,AE,MAE,MAPE,MSE,RMSE,R_adj
Default_reg,-0.01970584,0.4215968,0.03121151,0.3839392,0.6196283,0.6113912
Forward,0.00000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000
Backward,0.00000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000
Stepwise,0.00000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000


## 1.2 Forward Selection

In [11]:
####Forward####
lm.null_1=lm(user_rating_scaled~1,data=reg_1_train)
for_1=step(lm.null_1,direction="forward",scope=formula(lm.fit_1))
f.for_1=lm(formula(for_1),data=reg_1_train)
#reg_1 predict
p.for_1=predict(f.for_1,newdata=reg_1_test)
###6Measure###

#average error
ave.err.for_1=mean(reg_1_test$user_rating_scaled-p.for_1)
c_data1.mat[2,1]=ave.err.for_1
#mae
mae.for_1=mean(abs(reg_1_test$user_rating_scaled-p.for_1))
c_data1.mat[2,2]=mae.for_1
#mape
mape.for_1=0.01*mean(abs(reg_1_test$user_rating_scaled-p.for_1)/abs(reg_1_test$user_rating_scaled))
c_data1.mat[2,3]=mape.for_1
#mse
mse.for_1=mean((reg_1_test$user_rating_scaled-p.for_1)^2)
c_data1.mat[2,4]=mse.for_1
#rmse
rmse.for_1=sqrt(mse.for_1)
c_data1.mat[2,5]=rmse.for_1
#adj.r.squared
adj.for_1=summary(for_1)$adj.r.squared
c_data1.mat[2,6]=adj.for_1

Start:  AIC=20.07
user_rating_scaled ~ 1

                   Df Sum of Sq    RSS     AIC
+ user_rating_ver   1    3439.2 2314.9 -5200.8
+ ipadSc_urls.num   1     418.8 5335.3  -411.4
+ lang.num          1     167.7 5586.4  -147.6
+ cont_rating       3     166.4 5587.7  -142.3
+ prime_genre       1      81.8 5672.3   -60.1
+ rating_count_tot  1      37.9 5716.2   -15.8
+ size_mega         1      35.4 5718.7   -13.3
+ ver               1      27.0 5727.1    -4.9
+ rating_count_ver  1      24.6 5729.5    -2.5
+ price             1      10.2 5743.9    11.9
+ sup_devices.num   1       9.0 5745.1    13.1
<none>                          5754.1    20.1

Step:  AIC=-5200.83
user_rating_scaled ~ user_rating_ver

                   Df Sum of Sq    RSS     AIC
+ ver               1    48.156 2266.7 -5319.4
+ ipadSc_urls.num   1    20.816 2294.1 -5250.6
+ lang.num          1     6.896 2308.0 -5215.9
+ price             1     5.453 2309.4 -5212.4
+ sup_devices.num   1     5.371 2309.5 -5212.2
+ prim

In [12]:
c_data1.mat

,AE,MAE,MAPE,MSE,RMSE,R_adj
Default_reg,-0.01970584,0.4215968,0.03121151,0.3839392,0.6196283,0.6113912
Forward,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592
Backward,0.00000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000
Stepwise,0.00000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000


## 1.3 backward elimination

In [13]:
####Backward####
back_1=step(lm.fit_1,direction="backward")
f.back_1=lm(formula(back_1),data=reg_1_train)
#reg_1 predict
p.back_1=predict(f.back_1,newdata=reg_1_test)


#average error
ave.err.back_1=mean(reg_1_test$user_rating_scaled-p.back_1)
c_data1.mat[3,1]=ave.err.back_1
#mae
mae.back_1=mean(abs(reg_1_test$user_rating_scaled-p.back_1))
c_data1.mat[3,2]=mae.back_1
#mape
mape.back_1=0.01*mean(abs(reg_1_test$user_rating_scaled-p.back_1)/abs(reg_1_test$user_rating_scaled))
c_data1.mat[3,3]=mape.back_1
#mse
mse.back_1=mean((reg_1_test$user_rating_scaled-p.back_1)^2)
c_data1.mat[3,4]=mse.back_1
#rmse
rmse.back_1=sqrt(mse.back_1)
c_data1.mat[3,5]=rmse.back_1
#adj.r.squared
adj.back_1=summary(back_1)$adj.r.squared
c_data1.mat[3,6]=adj.back_1


Start:  AIC=-5388.51
user_rating_scaled ~ price + rating_count_tot + rating_count_ver + 
    user_rating_ver + ver + cont_rating + prime_genre + sup_devices.num + 
    ipadSc_urls.num + lang.num + size_mega

                   Df Sum of Sq    RSS     AIC
- size_mega         1      0.00 2231.0 -5390.5
- prime_genre       1      0.02 2231.0 -5390.5
- rating_count_tot  1      0.06 2231.1 -5390.4
- rating_count_ver  1      0.45 2231.5 -5389.3
<none>                          2231.0 -5388.5
- cont_rating       3      2.42 2233.4 -5388.3
- sup_devices.num   1      1.18 2232.2 -5387.5
- lang.num          1      2.27 2233.3 -5384.7
- price             1      3.68 2234.7 -5381.1
- ipadSc_urls.num   1     18.61 2249.6 -5342.9
- ver               1     40.99 2272.0 -5286.1
- user_rating_ver   1   2784.66 5015.7  -743.7

Step:  AIC=-5390.51
user_rating_scaled ~ price + rating_count_tot + rating_count_ver + 
    user_rating_ver + ver + cont_rating + prime_genre + sup_devices.num + 
    ipadSc_urls.n

In [14]:
c_data1.mat

,AE,MAE,MAPE,MSE,RMSE,R_adj
Default_reg,-0.01970584,0.4215968,0.03121151,0.3839392,0.6196283,0.6113912
Forward,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592
Backward,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592
Stepwise,0.00000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000


## 1.4 stepwise 

In [15]:
####Stepwise####
lm.null_1=lm(user_rating_scaled~1,data=reg_1_train)
step_1=step(lm.null_1,direction="both",scope=formula(lm.fit_1))
f.step_1=lm(formula(step_1),data=reg_1_train)
#reg_1 predict
p.step_1=predict(f.step_1,newdata=reg_1_test)


#average error
ave.err.step_1=mean(reg_1_test$user_rating_scaled-p.step_1)
c_data1.mat[4,1]=ave.err.step_1
#mae
mae.step_1=mean(abs(reg_1_test$user_rating_scaled-p.step_1))
c_data1.mat[4,2]=mae.step_1
#mape
mape.step_1=0.01*mean(abs(reg_1_test$user_rating_scaled-p.step_1)/abs(reg_1_test$user_rating_scaled))
c_data1.mat[4,3]=mape.step_1
#mse
mse.step_1=mean((reg_1_test$user_rating_scaled-p.step_1)^2)
c_data1.mat[4,4]=mse.step_1
#rmse
rmse.step_1=sqrt(mse.step_1)
c_data1.mat[4,5]=rmse.step_1
#adj.r.squared
adj.step_1=summary(step_1)$adj.r.squared
c_data1.mat[4,6]=adj.step_1


Start:  AIC=20.07
user_rating_scaled ~ 1

                   Df Sum of Sq    RSS     AIC
+ user_rating_ver   1    3439.2 2314.9 -5200.8
+ ipadSc_urls.num   1     418.8 5335.3  -411.4
+ lang.num          1     167.7 5586.4  -147.6
+ cont_rating       3     166.4 5587.7  -142.3
+ prime_genre       1      81.8 5672.3   -60.1
+ rating_count_tot  1      37.9 5716.2   -15.8
+ size_mega         1      35.4 5718.7   -13.3
+ ver               1      27.0 5727.1    -4.9
+ rating_count_ver  1      24.6 5729.5    -2.5
+ price             1      10.2 5743.9    11.9
+ sup_devices.num   1       9.0 5745.1    13.1
<none>                          5754.1    20.1

Step:  AIC=-5200.83
user_rating_scaled ~ user_rating_ver

                   Df Sum of Sq    RSS     AIC
+ ver               1      48.2 2266.7 -5319.4
+ ipadSc_urls.num   1      20.8 2294.1 -5250.6
+ lang.num          1       6.9 2308.0 -5215.9
+ price             1       5.5 2309.4 -5212.4
+ sup_devices.num   1       5.4 2309.5 -5212.2
+ prim

In [16]:
c_data1.mat

,AE,MAE,MAPE,MSE,RMSE,R_adj
Default_reg,-0.01970584,0.4215968,0.03121151,0.3839392,0.6196283,0.6113912
Forward,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592
Backward,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592
Stepwise,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592


# 2. result

- 모든 measure에 대하여 변수 선택기법을 적용한 경우 같은 수치가 나왔다.

- 그 이유는 3개의 모델이 모두 같은 변수를 선택했기 때문이다.

- user_rating_ver, ver, ipadSc_urls.num, price, lang.num, cont_rating~

In [17]:
f.for_1


Call:
lm(formula = formula(for_1), data = reg_1_train)

Coefficients:
    (Intercept)  user_rating_ver            ver.L  ipadSc_urls.num  
      -1.292936         0.414882         0.135269         0.031592  
          price         lang.num  sup_devices.num   cont_rating17+  
       0.004163         0.002730        -0.003973        -0.078198  
  cont_rating4+    cont_rating9+  
      -0.034431        -0.002569  


In [18]:
f.back_1


Call:
lm(formula = formula(back_1), data = reg_1_train)

Coefficients:
    (Intercept)            price  user_rating_ver            ver.L  
      -1.292936         0.004163         0.414882         0.135269  
 cont_rating17+    cont_rating4+    cont_rating9+  sup_devices.num  
      -0.078198        -0.034431        -0.002569        -0.003973  
ipadSc_urls.num         lang.num  
       0.031592         0.002730  


In [19]:
f.step_1


Call:
lm(formula = formula(step_1), data = reg_1_train)

Coefficients:
    (Intercept)  user_rating_ver            ver.L  ipadSc_urls.num  
      -1.292936         0.414882         0.135269         0.031592  
          price         lang.num  sup_devices.num   cont_rating17+  
       0.004163         0.002730        -0.003973        -0.078198  
  cont_rating4+    cont_rating9+  
      -0.034431        -0.002569  


# 3.   Data 2

## prime_genre : 

### 'Education' 'Entertainment' 'Games' 'Photo & Video' 'Utilities' 'etc'


- 6measure

      Average Error, MAE(Mean Absolute Error), 
      MAPE(Mean Absolute Percentage Error), MSE, RMSE, R^2_adj
      

- data : reg_2_train & reg_2_test      

- rating_count_ver, sum_rate_ver은 상관계수와 VIF가 높게 나와 제거했다

In [22]:
load('reg_2_train.rda')
load('reg_2_test.rda')
levels(reg_2_train$prime_genre)

[1] "Education"     "Entertainment" "Games"         "Photo & Video"
[5] "Utilities"     "etc"

In [23]:
drops=c("sum_rate_ver")
reg_2_train=reg_2_train[ , !(names(reg_2_train) %in% drops)]
reg_2_test=reg_2_test[ , !(names(reg_2_test) %in% drops)]
reg_2_train<-droplevels(reg_2_train)
reg_2_test <- droplevels(reg_2_test)
# 코드재사용을 위해 이름 덮어쓰기
reg_1_train = reg_2_train
reg_1_test = reg_2_test

## 3.1 linear-regression

In [24]:
c_data2.mat = matrix(0,4,6)
colnames(c_data2.mat) = c( "AE","MAE","MAPE","MSE","RMSE","R_adj") 
rownames(c_data2.mat) = c("Default_reg","Forward","Backward",'Stepwise')
c_data2.mat

,AE,MAE,MAPE,MSE,RMSE,R_adj
Default_reg,0,0,0,0,0,0
Forward,0,0,0,0,0,0
Backward,0,0,0,0,0,0
Stepwise,0,0,0,0,0,0


In [25]:
####Default####
set.seed(1)
lm.fit_1=lm(user_rating_scaled~.,data=reg_1_train)
#reg_1 predict
set.seed(1)
p.df_1=predict(lm.fit_1,newdata=reg_1_test)
###6Measure###
#average error
ave.err.df_1=mean(reg_1_test$user_rating_scaled-p.df_1)
c_data2.mat[1,1]=ave.err.df_1
#mae
mae.df_1=mean(abs(reg_1_test$user_rating_scaled-p.df_1))
c_data2.mat[1,2]=mae.df_1
#mape
mape.df_1=0.01*mean(abs(reg_1_test$user_rating_scaled-p.df_1)/abs(reg_1_test$user_rating_scaled))
c_data2.mat[1,3]=mape.df_1
#mse
mse.df_1=mean((reg_1_test$user_rating_scaled-p.df_1)^2)
c_data2.mat[1,4]=mse.df_1
#rmse
rmse.df_1=sqrt(mse.df_1)
c_data2.mat[1,5]=rmse.df_1
#adj.r.squared
adj.df_1=summary(lm.fit_1)$adj.r.squared
c_data2.mat[1,6]=adj.df_1
c_data2.mat

,AE,MAE,MAPE,MSE,RMSE,R_adj
Default_reg,-0.01969412,0.4216642,0.03119029,0.3841884,0.6198293,0.6112671
Forward,0.00000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000
Backward,0.00000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000
Stepwise,0.00000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000


## 3.2 Forward Selection

In [26]:
####Forward####
lm.null_1=lm(user_rating_scaled~1,data=reg_1_train)
for_1=step(lm.null_1,direction="forward",scope=formula(lm.fit_1))
f.for_1=lm(formula(for_1),data=reg_1_train)
#reg_1 predict
p.for_1=predict(f.for_1,newdata=reg_1_test)
###6Measure###

#average error
ave.err.for_1=mean(reg_1_test$user_rating_scaled-p.for_1)
c_data2.mat[2,1]=ave.err.for_1
#mae
mae.for_1=mean(abs(reg_1_test$user_rating_scaled-p.for_1))
c_data2.mat[2,2]=mae.for_1
#mape
mape.for_1=0.01*mean(abs(reg_1_test$user_rating_scaled-p.for_1)/abs(reg_1_test$user_rating_scaled))
c_data2.mat[2,3]=mape.for_1
#mse
mse.for_1=mean((reg_1_test$user_rating_scaled-p.for_1)^2)
c_data2.mat[2,4]=mse.for_1
#rmse
rmse.for_1=sqrt(mse.for_1)
c_data2.mat[2,5]=rmse.for_1
#adj.r.squared
adj.for_1=summary(for_1)$adj.r.squared
c_data2.mat[2,6]=adj.for_1

Start:  AIC=20.07
user_rating_scaled ~ 1

                   Df Sum of Sq    RSS     AIC
+ user_rating_ver   1    3439.2 2314.9 -5200.8
+ ipadSc_urls.num   1     418.8 5335.3  -411.4
+ lang.num          1     167.7 5586.4  -147.6
+ cont_rating       3     166.4 5587.7  -142.3
+ prime_genre       5     107.2 5646.9   -77.8
+ rating_count_tot  1      37.9 5716.2   -15.8
+ size_mega         1      35.4 5718.7   -13.3
+ ver               1      27.0 5727.1    -4.9
+ rating_count_ver  1      24.6 5729.5    -2.5
+ price             1      10.2 5743.9    11.9
+ sup_devices.num   1       9.0 5745.1    13.1
<none>                          5754.1    20.1

Step:  AIC=-5200.83
user_rating_scaled ~ user_rating_ver

                   Df Sum of Sq    RSS     AIC
+ ver               1    48.156 2266.7 -5319.4
+ ipadSc_urls.num   1    20.816 2294.1 -5250.6
+ lang.num          1     6.896 2308.0 -5215.9
+ price             1     5.453 2309.4 -5212.4
+ sup_devices.num   1     5.371 2309.5 -5212.2
+ prim

In [27]:
c_data2.mat

,AE,MAE,MAPE,MSE,RMSE,R_adj
Default_reg,-0.01969412,0.4216642,0.03119029,0.3841884,0.6198293,0.6112671
Forward,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592
Backward,0.00000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000
Stepwise,0.00000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000


## 3.3 backward elimination

In [28]:
####Backward####
back_1=step(lm.fit_1,direction="backward")
f.back_1=lm(formula(back_1),data=reg_1_train)
#reg_1 predict
p.back_1=predict(f.back_1,newdata=reg_1_test)


#average error
ave.err.back_1=mean(reg_1_test$user_rating_scaled-p.back_1)
c_data2.mat[3,1]=ave.err.back_1
#mae
mae.back_1=mean(abs(reg_1_test$user_rating_scaled-p.back_1))
c_data2.mat[3,2]=mae.back_1
#mape
mape.back_1=0.01*mean(abs(reg_1_test$user_rating_scaled-p.back_1)/abs(reg_1_test$user_rating_scaled))
c_data2.mat[3,3]=mape.back_1
#mse
mse.back_1=mean((reg_1_test$user_rating_scaled-p.back_1)^2)
c_data2.mat[3,4]=mse.back_1
#rmse
rmse.back_1=sqrt(mse.back_1)
c_data2.mat[3,5]=rmse.back_1
#adj.r.squared
adj.back_1=summary(back_1)$adj.r.squared
c_data2.mat[3,6]=adj.back_1


Start:  AIC=-5382.69
user_rating_scaled ~ price + rating_count_tot + rating_count_ver + 
    user_rating_ver + ver + cont_rating + prime_genre + sup_devices.num + 
    ipadSc_urls.num + lang.num + size_mega

                   Df Sum of Sq    RSS     AIC
- prime_genre       5      0.87 2231.0 -5390.5
- size_mega         1      0.00 2230.2 -5384.7
- rating_count_tot  1      0.05 2230.2 -5384.6
- rating_count_ver  1      0.46 2230.6 -5383.5
<none>                          2230.2 -5382.7
- cont_rating       3      2.39 2232.6 -5382.5
- sup_devices.num   1      1.15 2231.3 -5381.7
- lang.num          1      2.12 2232.3 -5379.2
- price             1      3.62 2233.8 -5375.4
- ipadSc_urls.num   1     18.77 2248.9 -5336.6
- ver               1     38.50 2268.7 -5286.5
- user_rating_ver   1   2767.61 4997.8  -756.2

Step:  AIC=-5390.45
user_rating_scaled ~ price + rating_count_tot + rating_count_ver + 
    user_rating_ver + ver + cont_rating + sup_devices.num + ipadSc_urls.num + 
    lang.num 

In [29]:
c_data2.mat

,AE,MAE,MAPE,MSE,RMSE,R_adj
Default_reg,-0.01969412,0.4216642,0.03119029,0.3841884,0.6198293,0.6112671
Forward,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592
Backward,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592
Stepwise,0.00000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000


## 3.4 stepwise 

In [30]:
####Stepwise####
lm.null_1=lm(user_rating_scaled~1,data=reg_1_train)
step_1=step(lm.null_1,direction="both",scope=formula(lm.fit_1))
f.step_1=lm(formula(step_1),data=reg_1_train)
#reg_1 predict
p.step_1=predict(f.step_1,newdata=reg_1_test)


#average error
ave.err.step_1=mean(reg_1_test$user_rating_scaled-p.step_1)
c_data2.mat[4,1]=ave.err.step_1
#mae
mae.step_1=mean(abs(reg_1_test$user_rating_scaled-p.step_1))
c_data2.mat[4,2]=mae.step_1
#mape
mape.step_1=0.01*mean(abs(reg_1_test$user_rating_scaled-p.step_1)/abs(reg_1_test$user_rating_scaled))
c_data2.mat[4,3]=mape.step_1
#mse
mse.step_1=mean((reg_1_test$user_rating_scaled-p.step_1)^2)
c_data2.mat[4,4]=mse.step_1
#rmse
rmse.step_1=sqrt(mse.step_1)
c_data2.mat[4,5]=rmse.step_1
#adj.r.squared
adj.step_1=summary(step_1)$adj.r.squared
c_data2.mat[4,6]=adj.step_1


Start:  AIC=20.07
user_rating_scaled ~ 1

                   Df Sum of Sq    RSS     AIC
+ user_rating_ver   1    3439.2 2314.9 -5200.8
+ ipadSc_urls.num   1     418.8 5335.3  -411.4
+ lang.num          1     167.7 5586.4  -147.6
+ cont_rating       3     166.4 5587.7  -142.3
+ prime_genre       5     107.2 5646.9   -77.8
+ rating_count_tot  1      37.9 5716.2   -15.8
+ size_mega         1      35.4 5718.7   -13.3
+ ver               1      27.0 5727.1    -4.9
+ rating_count_ver  1      24.6 5729.5    -2.5
+ price             1      10.2 5743.9    11.9
+ sup_devices.num   1       9.0 5745.1    13.1
<none>                          5754.1    20.1

Step:  AIC=-5200.83
user_rating_scaled ~ user_rating_ver

                   Df Sum of Sq    RSS     AIC
+ ver               1      48.2 2266.7 -5319.4
+ ipadSc_urls.num   1      20.8 2294.1 -5250.6
+ lang.num          1       6.9 2308.0 -5215.9
+ price             1       5.5 2309.4 -5212.4
+ sup_devices.num   1       5.4 2309.5 -5212.2
+ prim

In [31]:
c_data2.mat

,AE,MAE,MAPE,MSE,RMSE,R_adj
Default_reg,-0.01969412,0.4216642,0.03119029,0.3841884,0.6198293,0.6112671
Forward,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592
Backward,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592
Stepwise,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592


# 4. result

- 모든 measure에 대하여 변수 선택기법을 적용한 경우 같은 수치가 나왔다.

- 그 이유는 3개의 모델이 모두 같은 변수를 선택했기 때문이다.

- user_rating_ver, ver, ipadSc_urls.num, price, lang.num, cont_rating~

In [32]:
f.for_1


Call:
lm(formula = formula(for_1), data = reg_1_train)

Coefficients:
    (Intercept)  user_rating_ver            ver.L  ipadSc_urls.num  
      -1.292936         0.414882         0.135269         0.031592  
          price         lang.num  sup_devices.num   cont_rating17+  
       0.004163         0.002730        -0.003973        -0.078198  
  cont_rating4+    cont_rating9+  
      -0.034431        -0.002569  


In [33]:
f.back_1


Call:
lm(formula = formula(back_1), data = reg_1_train)

Coefficients:
    (Intercept)            price  user_rating_ver            ver.L  
      -1.292936         0.004163         0.414882         0.135269  
 cont_rating17+    cont_rating4+    cont_rating9+  sup_devices.num  
      -0.078198        -0.034431        -0.002569        -0.003973  
ipadSc_urls.num         lang.num  
       0.031592         0.002730  


In [34]:
f.step_1


Call:
lm(formula = formula(step_1), data = reg_1_train)

Coefficients:
    (Intercept)  user_rating_ver            ver.L  ipadSc_urls.num  
      -1.292936         0.414882         0.135269         0.031592  
          price         lang.num  sup_devices.num   cont_rating17+  
       0.004163         0.002730        -0.003973        -0.078198  
  cont_rating4+    cont_rating9+  
      -0.034431        -0.002569  


# 5. final_result

In [35]:
c_data1.mat

,AE,MAE,MAPE,MSE,RMSE,R_adj
Default_reg,-0.01970584,0.4215968,0.03121151,0.3839392,0.6196283,0.6113912
Forward,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592
Backward,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592
Stepwise,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592


In [36]:
c_data2.mat

,AE,MAE,MAPE,MSE,RMSE,R_adj
Default_reg,-0.01969412,0.4216642,0.03119029,0.3841884,0.6198293,0.6112671
Forward,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592
Backward,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592
Stepwise,-0.02012952,0.4215807,0.03124251,0.3840035,0.6196801,0.6115592
